In [1]:
import numpy as np
from pyapril.detector import cc_detector_ons
from pyapril import clutterCancellation as CC
from pyapril.caCfar import CA_CFAR
from pyapril.RDTools import export_rd_matrix_img

# Import libraries to make gif from plot images
from glob import glob
import contextlib
from PIL import Image
from IPython.display import clear_output
from os import remove

In [2]:
# Paramaters
# Follow working directory format, with "pyApril test.ipynb" + "Data", "Images", and "GIFs" folders in the working folder
# Remember to move old gifs out of ./GIFS/ if the name is same as new one, otherwise it will be overwritten

# Signal parameters - should match expected parameters of recorded data
sample_rate = 3 * 10**6  # sample rate
max_bistatic_range = 2**7 # should be power of 2 to work with cc_detector_ons
max_doppler = 250 # should be divisible by 10 to work with export_rd_matrix_img
N = 2**21 # samples to use in cross correlation, should be power of 2, 19-22 work well at 3-6 Msamples/s

# Iteration parameters
offset = 0 # offset from start of data, set in number of samples to trim from start of data
interval = int(10 * 10**5) # interval length in number of samples, 5*10**5 or 1*10**6 seem to work well for gif output
dyn_range = 10 * np.log10(N) # don't touch, used for plotting function's colormap
td_filter_dimension = max_bistatic_range # don't touch, used for CC.Wiener_SMI_MRE

# CFAR parameters - adjust to produce best result from data
win_param = [10, 10, 5, 5] # tried various CFAR parameters, fairly selective and seems to produce decent results
threshold = 12

# Edit gif name as needed
gif_name = "s" + (str(int(np.log2(N))))+ "_r" + str(max_bistatic_range) + "_d" + str(max_doppler)

print(f"Parameters:\nMax Bistatic Range: {max_bistatic_range} (km)\nMax Doppler Frequency: {max_doppler} (Hz)")
print(f"Sample Length: {N/sample_rate:.4} (s)\nIteration interval: {interval/sample_rate:.4} (s)")

Parameters:
Max Bistatic Range: 128 (km)
Max Doppler Frequency: 250 (Hz)
Sample Length: 0.6991 (s)
Iteration interval: 0.3333 (s)


In [6]:
# Data read from ./Data/ - ref_ch should be named rx1.bin, surv_ch should be named rx2.bin
ref_ch = np.fromfile("./Data/rx1.bin", np.complex64)

surv_ch = np.fromfile("./Data/rx2.bin", np.complex64, count=ref_ch.size)

print(f"ref_ch length: {ref_ch.size/sample_rate} (s)") # N samples / sampling rate = sample time
print(f"surv_ch length: {surv_ch.size/sample_rate} (s)")

# print("Performing clutter cancellation...") # comment this block to disable clutter cancellation
# try: # save filtered surv_ch to file to allow future runs on dataset to save time on cc
#     surv_ch = np.fromfile("./Data/rx2_cc.bin", np.complex64, count=ref_ch.size)
#     print("surv_ch opened from rx2_cc.bin")
# except:
#     print("No rx2_cc.bin file was found, filtering surv_ch")
#     surv_ch, w = CC.Wiener_SMI_MRE(ref_ch, surv_ch, td_filter_dimension)
#     np.save("./Data/rx2_cc.bin",surv_ch)
#     print("Filtered surv_ch saved to rx2_cc.bin")
# gif_name += "_cc"
# print("Clutter cancellation complete")

ref_ch length: 13.588474666666666 (s)
surv_ch length: 13.588474666666666 (s)


In [ ]:
# clearing previous images from ./Images/
files = glob('./Images/*.png')
for f in files:
    remove(f)

# While loop to iterate over length of data recorded in intervals and export rd plot images
i = 1
n_frames = int(np.floor((ref_ch.size - N - offset)/interval) + 1) # number of frames to divide total data into
while i <= n_frames:
    print(f"-----Image frame {i} out of {n_frames}-----")

    ref_ch_sample = ref_ch[offset : offset + N]
    surv_ch_sample = surv_ch[offset : offset + N]
    print(f"Sample period: {offset/sample_rate:.4} (s) - {(offset+N)/sample_rate:.4} (s)")

    # Cross-correlation detector, don't modify parameters here
    rd_matrix = cc_detector_ons(
        ref_ch_sample,
        surv_ch_sample,
        sample_rate,
        max_doppler,
        max_bistatic_range,
    )
    print("Calculated rd_matrix")

    # if i==1: # Comment this block to disable CFAR
    #     CFAR = CA_CFAR(win_param, threshold, rd_size=rd_matrix.shape)
    #     gif_name += "_cfar" # adding cfar parameter to end of gif name
    # rd_matrix = CFAR(rd_matrix)
    # print("CFAR complete")

    export_rd_matrix_img(
        fname= "./Images/" + str(i) + ".png",
        rd_matrix=rd_matrix,
        max_Doppler=max_doppler,
        ref_point_range_index=0,
        ref_point_Doppler_index=0,
        box_color="None",
        box_size=0,
        dyn_range=dyn_range,
        dpi=200,
        interpolation="sinc",
        cmap="jet",
    )
    print("Saved image\n")

    offset += interval
    i += 1
    
    clear_output()

# Making GIF from pngs
print("Saving GIF...\n")
with contextlib.ExitStack() as stack:
    imgs = (stack.enter_context(Image.open(f)) for f in sorted(glob("./Images/*.png"), key = lambda fname: int(fname[9:-4])))
    img = next(imgs)
    img.save(fp="./GIFs/" + gif_name + ".gif", format="GIF", append_images=imgs, save_all=True, duration=100, loop=0)
    print(f"GIF Saved as {gif_name + '.gif'}")